In [58]:
:dep ndarray
:dep ndarray-interp
:dep vawt = {path="../"}
:dep csv
:dep ndarray-csv
:dep plotters = {default_features = false, features = ["evcxr", "all_series"]}

use std::fs::File;
use std::f64::consts::{FRAC_PI_2, PI};
use std::error::Error;

use csv::ReaderBuilder;
use ndarray::{array, Array2, Axis, Array};
use ndarray_csv::Array2Reader;
use plotters::prelude::*;

use vawt::areofoil::Aerofoil;

In [59]:
fn read_array(path: &str) -> Result<Array2<f64>, Box<dyn Error>> {
    let file = File::open(path)?;
    let mut reader = ReaderBuilder::new()
        .has_headers(false)
        .trim(csv::Trim::All)
        .delimiter(b',')
        .from_reader(file);

    let mut arr: Array2<f64> = reader.deserialize_array2_dynamic().unwrap();
    for mut datapoint in arr.axis_iter_mut(Axis(0)) {
        datapoint[0] = datapoint[0].to_radians();
    }
    Ok(arr)
}

In [67]:
let aerofoil = Aerofoil::builder()
    .add_data_row(read_array("NACA0018/NACA0018Re0080.data")?, 80_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0040.data")?, 40_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0160.data")?, 160_000.0)?
    .set_aspect_ratio(12.8)
    .update_aspect_ratio(true)
    .symmetric(true)
    .build()?;

In [68]:
let x = Array::linspace(0.0, FRAC_PI_2, 100);
//let xy: impl Iter> = x.into_iter().map(|x|{aerofoil.cl_cd(x, 45_000.0)});
let arr = read_array("NACA0018/NACA0018Re0080.data")?;


evcxr_figure((640,480), |root| {    
    // Create a chart context with the specified ranges
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..0.5, 0f64..2f64)?;

    // Customize the x-axis labels
    chart.configure_mesh()
        .x_labels(8)
        .y_labels(5)
        .x_label_formatter(&|x| format!("{:.1}π", x))
        .draw()?;

    chart.draw_series(LineSeries::new(x.iter().map( |&x| (x / PI, aerofoil.cl_cd(x, 80_000.0).cl())), &RED))?;
    chart.draw_series(LineSeries::new(
        arr.axis_iter(Axis(0)).map(|a| (a[0] / PI, a[1])), &BLUE
    ))?;
    Ok(())
})

0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
 
0.0π
 
 
 
0.1π
 
 
 
0.2π
 
 
 
0.3π
 
 
 
0.4π
 
 
 
0.5π
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="40,439 46,424 52,406 58,388 64,371 70,354 76,339 82,324 88,310 94,301 100,295 106,295 112,299 118,302 124,303 130,305 136,305 142,305 148,305 154,305 161,304 167,303 173,302 179,301 185,299 191,298 197,297 203,295 209,294 215,293 221,291 227,290 233,289 239,288 245,286 251,285 257,284 263,284 269,283 275,282 282,282 288,281 294,281 300,280 306,280 312,280 318,280 324,280 330,281 336,281 342,282 348,283 354,283 360,284 366,285 372,287 378,288 384,289 390,291 396,293 403,295 409,296 415,299 421,301 427,303 433,306 439,308 445,311 451,314 457,317 463,320 469,323 475,326 481,329 487,333 493,336 499,340 505,344 511,347 517,351 524,355 530,359 536,363 542,367 548,372 554,376 560,380 566,385 572,389 578,394 584,398 590,403 596,407 602,412 608,416 614,421 620,426 626,430 632,435 639,439 "/>